Notebook

# Climate Displacement

Year
State
Households Inflow (Number of Returns)
Households Outflow (Number of Returns)
Individuals Inflow (Number of Exemptions)
Individuals Outflow (Number of Exemptions)
Chemical
Dam/Levee Break
Drought
Earthquake
Fire
Flood
Human Cause
Hurricane
Ice
Mud/Landslide
Other
Snow
Storm
Terrorism
Tornado
Tsunami
Typhoon
Volcano
Water
Winter

In [8]:
"""
Folders Setup

code 
    notebook.ipynb
    data
        Disasters
            FEMA_dataset.csv
        StateMigration
            1990to1991StateMigration
                 1990to1991StateMigrationInflow
                     Alabama91in.xls
                     Alaska91in.xls
                     .
                     .
                     .
                     Wisconsin91in.xls
                     Wyoming91in.xls 
                 1990to1991StateMigrationOutflow
                     Alabama91Out.xls
                     Alaska91Out.xls 
                     .
                     .
                     .
                     Wisconsin91Out.xls
                     Wyoming91Out.xls 
            1991to1992StateMigration
            1992to1993StateMigration
            1993to1994StateMigration
            1994to1995StateMigration
            1995to1996StateMigration
            1996to1997StateMigration
            1997to1998StateMigration
            1998to1999StateMigration
            1999to2000StateMigration
            2000to2001StateMigration
            2001to2002StateMigration
            2002to2003StateMigration
            2003to2004StateMigration
            2004to2005StateMigration
            2005to2006StateMigration
            2006to2007StateMigration
            2007to2008StateMigration
            2008to2009StateMigration
            2009to2010StateMigration
            2010to2011StateMigration 
"""

'\nFolders Setup\n\ncode \n    notebook.ipynb\n    data\n        Disasters\n            FEMA_dataset.csv\n        StateMigration\n            1990to1991StateMigration\n                 1990to1991StateMigrationInflow\n                     Alabama91in.xls\n                     Alaska91in.xls\n                     .\n                     .\n                     .\n                     Wisconsin91in.xls\n                     Wyoming91in.xls \n                 1990to1991StateMigrationOutflow\n                     Alabama91Out.xls\n                     Alaska91Out.xls \n                     .\n                     .\n                     .\n                     Wisconsin91Out.xls\n                     Wyoming91Out.xls \n            1991to1992StateMigration\n            1992to1993StateMigration\n            1993to1994StateMigration\n            1994to1995StateMigration\n            1995to1996StateMigration\n            1996to1997StateMigration\n            1997to1998StateMigration\n          

In [9]:
"""
Pseudo Code:

FEMA Dataset Pre-Processing 
1. Create new FEMA_dataset with columns "Year",  "State" and "Disaster Type"
    - contains Year, State, Disaster Type
2. Name file "State_Disasters_by_Year"

StateMigration Data Pre-Processing
1. Convert all datasets in StateMigration from .xls into .csv files
2. Extract "Total Flow" row with "Number of Returns" and "Number of Exemptions" - assign I if from inflow and O if from outflow - from every state file.
3. Extract "State" and "Year" from every file
4. Create file with "State" (from file name), "Year" (from file name), "Number_of_Returns_I", "Number_of_Exemptions_I", "Number_of_Returns_I" and "Number_of_Exemptions_O"
5. Name file "State Migration by Year"

Merge Datasets
1. Merge datasets on common attributes "Year" and "State"
2. Name dataset "State_Migration_and_Disasters_by_Year"

Question
- Where is 2003to2004 MigrationOutflow data?


read_file = pd.read_excel ("Test.xlsx")
 
# Write the dataframe object
# into csv file
read_file.to_csv ("Test.csv",
                  index = None,
                  header=True)
"""

'\nPseudo Code:\n\nFEMA Dataset Pre-Processing \n1. Create new FEMA_dataset with columns "Year",  "State" and "Disaster Type"\n    - contains Year, State, Disaster Type\n2. Name file "State_Disasters_by_Year"\n\nStateMigration Data Pre-Processing\n1. Convert all datasets in StateMigration from .xls into .csv files\n2. Extract "Total Flow" row with "Number of Returns" and "Number of Exemptions" - assign I if from inflow and O if from outflow - from every state file.\n3. Extract "State" and "Year" from every file\n4. Create file with "State" (from file name), "Year" (from file name), "Number_of_Returns_I", "Number_of_Exemptions_I", "Number_of_Returns_I" and "Number_of_Exemptions_O"\n5. Name file "State Migration by Year"\n\nMerge Datasets\n1. Merge datasets on common attributes "Year" and "State"\n2. Name dataset "State_Migration_and_Disasters_by_Year"\n\nQuestion\n- Where is 2003to2004 MigrationOutflow data?\n\n\nread_file = pd.read_excel ("Test.xlsx")\n \n# Write the dataframe object\n

In [10]:
import shutil
import pandas as pd

"""
FEMA Dataset Pre-Processing

Creates State_Disasters_by_Year.csv with:
- State
- Disaster Type
- Start Year
- End Year
"""

# copy original FEMA dataset to new file
original = r'../code/data/Disasters/FEMA_dataset.csv'
new = r'../code/data/Disasters/State_Disasters_by_Year.csv'
shutil.copyfile(original, new)

# read csv file
data = pd.read_csv('../code/data/Disasters/State_Disasters_by_Year.csv')

# delete irrelevant rows
data.pop('Declaration Number')
data.pop('Declaration Type')
data.pop('Declaration Date')
data.pop('County')
data.pop('Disaster Title')
data.pop('Close Date')
data.pop('Individual Assistance Program')
data.pop('Public Assistance Program')
data.pop('Hazard Mitigation Program')
data.pop('Individuals & Households Program')

# extract years
data['Start Year'] = pd.DatetimeIndex(data['Start Date']).year
data['End Year'] = pd.DatetimeIndex(data['End Date']).year

# delete start and end dates
data.pop('Start Date')
data.pop('End Date')

print(data)

# save changes csv
data.to_csv('../code/data/Disasters/State_Disasters_by_Year.csv')

      State Disaster Type  Start Year  End Year
0        GA       Tornado        1953    1953.0
1        TX       Tornado        1953    1953.0
2        LA         Flood        1953    1953.0
3        MI       Tornado        1953    1953.0
4        MT         Flood        1953    1953.0
...     ...           ...         ...       ...
46180    CA         Storm        2017    2017.0
46181    CA         Storm        2017    2017.0
46182    CA         Storm        2017    2017.0
46183    CA         Storm        2017    2017.0
46184    CA         Storm        2017    2017.0

[46185 rows x 4 columns]


In [11]:
import os
import glob

# Convert all files in StateMigration folder from .xls to .csv

# create list of xls files
xls_list = glob.glob("../code/data/StateMigration/*/*/*.xls")

# replace xls 
for xls_file in xls_list:
    read_file = pd.read_table(xls_file)
    read_file.to_csv(xls_file[0:-3]+'csv', index = None, header=True)
    #read_xls
    #DataFrame
    #print to_csv
    
    os.remove(xls_file)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte

In [ ]:
"""
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        read_file = pd.read_excel (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.xls')
    read_file.to_csv (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.csv', index = None, header=True)


read_file = pd.read_excel (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.xls')
read_file.to_csv (r'../code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow/Alabama91in.csv', index = None, header=True)



location = "/Users/neely/Desktop/climate-displacement/code/data/StateMigration/1990to1991StateMigration/1990to1991StateMigrationInflow"
placement = "./Users/neely/Desktop/climate-displacement/code/data/StateMigration"

for file in glob.glob("*.xls"):
"""

"""
from os.path import isfile, join
onlyfiles = [f for f in listdir('../code/data/StateMigration/') if isfile(join('../code/data/StateMigration/', f))]
"""